```
This notebook sets up and runs a Cartesian analog to the Galewsky
2004 test case, defined in the paper 
"An initial-value problem for testing numerical models of the global
shallow-water equations"

Copyright (C) 2016, 2017, 2018  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
#%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
import scipy as scipy
from scipy import integrate
from matplotlib import animation, rc
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec

import os
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
import datetime
import sys

from importlib import reload

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../')))

#Set large figure sizes
#rc('figure', figsize=(16.0, 12.0))
#rc('animation', html='html5')
plt.rcParams["animation.html"] = "jshtml"

#Import our simulator
from SWESimulators import FBL, CTCS, KP07, CDKLM16, PlotHelper, Common, IPythonMagic
#Import initial condition and bathymetry generating functions:
from SWESimulators.BathymetryAndICs import *

In [ ]:
#%setup_logging --out galewsky.log
%cuda_context_handler gpu_ctx

# Galewsky 2004 test case

The test case is defined for spheric global shallow water models representing atmospheric flow, in the paper 
```
Joseph Galewsky, Richard K. Scott & Lorenzo M. Polvani (2004) An initial-
value problem for testing numerical models of the global shallow-water equations, Tellus A:
Dynamic Meteorology and Oceanography, 56:5, 429-440, DOI: 10.3402/tellusa.v56i5.14436
```
which can be found here: https://doi.org/10.3402/tellusa.v56i5.14436

The case describes a smooth yet moving around the globe initialized with a balanced $\eta$-field which describes a steady-state solution.
A perturbation is then added to the initial conditions, which generates turbulent solutions for the inviscid case, and a well-defined reference solution for the viscid case.

The **first goal** is therefore to rewrite the initial conditions to Cartesian coordinates, and reproduce the steady state solution. We here consider whether we should use the $\beta$-model, or if it is sufficient to consider the case with a constant Coriolis force.

The **second goal** is to add the perturbation and see if we can recognize the reference solution provided by Galewsky. It will be interesting to see whether CTCS is able to obtain this solution.

In [ ]:
# DEFINE PARAMETERS

sim_args = {
    "gpu_ctx": gpu_ctx,
    "nx": 500, "ny": 200,
    "dx": 10000.0, "dy": 10000.0,
    "dt": 20,
    "g": 9.81,
    "f": 1.2e-4,
    "coriolis_beta": 0.0,
    "r": 0.0,
    "boundary_conditions": Common.BoundaryConditions(1,2,1,2)
}

cdklm_args = {
    "rk_order": 2,
}

depth = 100
rossby_radius = np.sqrt(sim_args["g"]*depth)/sim_args["f"]
print(rossby_radius)

In [ ]:

def sim_animation(simulator, T, crossection=False, sub_dt=10):
    eta1, u1, v1 = sim.download(interior_domain_only=True)
    
    #Create figure and plot initial conditions
    fig = plt.figure(figsize=(16, 16))
    domain_extent = [0, sim.nx*sim.dx, 0, sim.ny*sim.dy]
    
    eta_max = eta1.max()*0.5
    hu_max = max(v1.max(), u1.max())
    
    ax_eta = plt.subplot(3,1,1)
    sp_eta = ax_eta.imshow(eta1, interpolation="none", origin='bottom', vmin=-eta_max, vmax=eta_max, extent=domain_extent)
    #ax_eta.colorbar()
    
    ax_u = plt.subplot(3,1,2)
    sp_u = ax_u.imshow(u1, interpolation="none", origin='bottom', vmin=-hu_max, vmax=hu_max, extent=domain_extent)
    #ax_u.colorbar()
    
    ax_v = plt.subplot(3,1,3)
    sp_v = ax_v.imshow(v1, interpolation="none", origin='bottom', vmin=-hu_max, vmax=hu_max, extent=domain_extent)
    #ax_v.colorbar()
    
    #Helper function which simulates and plots the solution
    def animate(i):
        if (i>0):
            t = sim.step(sub_dt*sim.dt)
        else:
            t = 0.0
        eta1, u1, v1 = sim.download(interior_domain_only=True)
        
        #Update plots
        fig.sca(ax_eta)
        sp_eta.set_data(eta1)
        
        fig.sca(ax_u)
        sp_u.set_data(u1)
        
        fig.sca(ax_v)
        sp_v.set_data(v1)
        
        fig.suptitle("Time = {:04.0f} s ({:s})".format(t, sim.__class__.__name__), fontsize=18)
        print(".", end='')

    #Matplotlib for creating an animation
    anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
    plt.close(fig)
    return anim

In [ ]:
ghosts = np.array([2,2,2,2]) # north, east, south, west
validDomain = np.array([2,2,2,2]) #

dataShape = (sim_args["ny"] + ghosts[0]+ghosts[2], 
             sim_args["nx"] + ghosts[1]+ghosts[3])
dataShapeHi = (sim_args["ny"] + ghosts[0]+ghosts[2]+1, 
             sim_args["nx"] + ghosts[1]+ghosts[3]+1)

eta = np.zeros(dataShape, dtype=np.float32, order='C');
hv  = np.zeros(dataShape, dtype=np.float32, order='C');
hu  = np.zeros(dataShape, dtype=np.float32, order='C');

center_x = (sim_args["nx"]+2)/2
center_y = 0


for j in range(0, sim_args["ny"]+4):
    for i in range(0,sim_args["nx"]+4):
        scale = 1
        #r_i = bump_size*(i-center_x)**2 *sim_args["dx"]**2
        #r_j = bump_size*(j-center_y) *sim_args["dy"]
        
        r = np.sqrt((i-center_x)**2 + (j-center_y)**2)*sim_args["dx"]
        
        #f_func = np.exp(-r_i/rossby_radius)
        
        eta[j,i] = np.exp(-scale*r/rossby_radius)
        #hu[ j,i] = (sim_args["g"]*depth/(sim_args["f"]*rossby_radius))*f_func
        

const = sim_args["g"]*depth/sim_args["f"]
for j in range(2,sim_args["ny"]+2):
    for i in range(2,sim_args["nx"]+2):
        #hv[j,i] =  const*(eta[j, i+1] - eta[j,i-1])/(2*sim_args["dx"])
        hu[j,i] = -const*(eta[j+1, i] - eta[j-1,i])/(2*sim_args["dy"])
        
                             
init_args = {
    "eta0" : eta,
    "hu0"  : hu,
    "hv0"  : hv,
    "H"    : np.ones(dataShapeHi, dtype=np.float32, order='C')*depth
}


sim = CDKLM16.CDKLM16(**sim_args, **init_args)

#sim_animation(sim, 100)
#sim_animation(sim, 1)

In [ ]:
ghosts = [1,1,1,1]
validDomain = np.array([1,1,1,1]) #

dataShape = (sim_args["ny"] + ghosts[0]+ghosts[2], 
             sim_args["nx"] + ghosts[1]+ghosts[3])

eta = np.zeros(dataShape, dtype=np.float32, order='C');
hu = np.zeros((dataShape[0], dataShape[1]+1), dtype=np.float32);
hv = np.zeros((dataShape[0]+1, dataShape[1]), dtype=np.float32);  

center_x = (sim_args["nx"]+2)/2
center_y = 0
y_mid = (sim_args["ny"]+2)/2

H = np.ones(eta.shape, dtype=np.float32, order='C')*depth


for j in range(0, sim_args["ny"]+2):
    for i in range(0,sim_args["nx"]+2):
        scale = 0.3
        
        r_j = np.sqrt((j-center_y)**2)*sim_args["dy"]
        r_i = np.sqrt((i-center_x)**2)*sim_args["dx"]
        
        
        f_func = np.exp(-scale*r_i/rossby_radius)
        
        eta[j,i] = np.exp(-r_j/rossby_radius)*f_func
        #eta[j,i] = f_func
        
        #tanh = -np.tanh(200*(y_mid - j)/sim_args['dy'])/2
        #H[j,i] = tanh*depth + depth
        #hu[ j,i] = (sim_args["g"]*depth/(sim_args["f"]*rossby_radius))*f_func
        

for j in range(0, sim_args["ny"]+2):
    for i in range(0,sim_args["nx"]+2):
        scale = 0.3
        
        r_j = np.sqrt((j-center_y)**2)*sim_args["dy"]
        
        
        #f_func = 1.0 - np.tanh((np.sqrt((i-center_x)**2) - 30)/10)
        f_func = 1.0 - np.tanh((np.sqrt((i-center_x)**2)*sim_args["dx"] - rossby_radius)/(rossby_radius/3))
        
        eta[j,i] = np.exp(-r_j/rossby_radius)*f_func
        #eta[j,i] = f_func
        
        #tanh = -np.tanh(200*(y_mid - j)/sim_args['dy'])/2
        #H[j,i] = tanh*depth + depth
        #hu[ j,i] = (sim_args["g"]*depth/(sim_args["f"]*rossby_radius))*f_func
        


for j in range(1,sim_args["ny"]+1):
    for i in range(1,sim_args["nx"]+2):
        hu[j,i] = -H[j,i]*sim_args["g"]/sim_args["f"]*(eta[j+1, i] - eta[j,i])/(sim_args["dy"])


init_args = {
    "eta0" : eta,
    "hu0"  : hu,
    "hv0"  : hv,
    "H"    : H,
    "A" : 0.0
}


#plt.plot(eta[2:,250])
#plt.plot(eta[2,:])
#plt.plot(eta[20,:])
#osidhgd

sim = CTCS.CTCS(**sim_args, **init_args)

sim_animation(sim, 100, sub_dt=400)
#sim_animation(sim, 1)

Beskrivelse av init + boundary conditions. Beskrivelse av valg av parametere sammenlignet med Rossbyradien.

Analytisk  fasehastighet, og plot som viser at den stemmer. HovmÃ¼ller diagram med karakteristikkene.

Vurdere Ã¥ si noe om energibetraktninger.
Sjekke at bÃ¸lgen er lik etter en "runde" (en periode). Fasehastighet er $\sqrt{gH}$.


In [ ]:
plt.plot(eta[2:,250])
plt.plot(eta[2,:])
plt.plot(eta[20,:])
eta2, hu2, hv2 = sim.download()
plt.plot(eta2[2,:])
plt.plot(eta2[20,:])

#plt.imshow(hv2, origin='lower')
#plt.colorbar()

fig=plt.figure(figsize=(15,15))
plt.imshow(hu2, origin='lower')
plt.colorbar()

fig=plt.figure(figsize=(15,15))
plt.imshow(eta, origin='lower')
plt.colorbar()

## Investigating the initial conditions based on spheric coordinates

In [ ]:
# global constants
OMEGA = 7.292e-5 # 1/s  - Angular rotation speed of the earth
GRAVITY = 9.80616 # m/s^2 - gravitational acceleration
RADIUS = 6.37122e6 # m - radius of the Earth
 
# parameters
u_max = 80 #m/s
phi_0 = np.pi/7.0
phi_1 = np.pi/2.0 - phi_0
e_n = np.exp( -4/(phi_1 - phi_0)**2)
h_0_guestimate = 10.4e3 # m  - global mean layer depth - estimated from the plot in Galewsky Fig. 1.

midpoint_phi = np.pi/4



# Different Coriolis models
def spheric_coriolis(lat):
    return 2*OMEGA*np.sin(lat)

def const_coriolis(lat):
    return 2*OMEGA*np.sin(midpoint_phi)*np.ones_like(lat)

def beta_coriolis(lat):
    return spheric_coriolis(phi_0) + (lat-phi_0)*(spheric_coriolis(phi_1) - spheric_coriolis(phi_0))/(phi_1 - phi_0)

# tangens corresponding to the beta model
def beta_tan(lat):
    return np.tan(phi_0) + (lat - phi_0)*(np.tan(phi_1) - np.tan(phi_0))/(phi_1 - phi_0)


# smooth step function
def smoothstep(x,loc, slope=1):
    return np.exp(slope*x)/(np.exp(loc*slope) + np.exp(slope*x))


# The initialization function for u
def init_u_scalar(lat):
    return (u_max/e_n) *np.exp(1/((lat-phi_0)*(lat-phi_1)))

def init_u(lat):
    steps = smoothstep(lat, phi_0, slope=50)*(1-smoothstep(lat, phi_1, slope=50))
    if np.isscalar(lat):
        return steps*init_u_scalar(lat)
    else:
        
        out = np.zeros_like(lat)
        for i in range(len(lat)):
            if lat[i] > phi_0 and lat[i] <= phi_1:
                out[i] = init_u_scalar(lat[i])
        return steps*out
    

# Integrand for initialization of h
def init_h_integrand(lat, coriolis_function, tan_function):
    return RADIUS*init_u(lat)*(coriolis_function(lat) + (tan_function(lat)/RADIUS)*init_u(lat))

phi = np.linspace(0, np.pi/2, num=100)
u0 = np.zeros_like(phi)
u0 = init_u(phi)

def generate_gh0(lat, coriolis_function, tan_function, lat_0, h_0):
    gh0 = np.zeros_like(lat)

    for i in range(lat.size):
        gh0[i] = GRAVITY*h_0 - scipy.integrate.quadrature(init_h_integrand, phi_0, lat[i], 
                                                          args=(coriolis_function, tan_function))[0]

    # For some wierd reason, the integration fails for the southmost value where the expression for
    # u is used.
    # This is a dirty fix:
    #for i in range(gh0.size):
    #    if np.isnan(gh0[i]):
    #        gh0[i] = 0.5*(gh0[i+1] + gh0[i-1])
    return gh0
gh0 = generate_gh0(phi, spheric_coriolis, np.tan, phi_0, h_0_guestimate)

# Look at the integrand for h
integrand = init_h_integrand(phi, spheric_coriolis, np.tan)


silly_integration = integrand.sum()*np.ones_like(integrand)
for i in range(1,len(integrand)):
    silly_integration[i] = silly_integration[i-1] - integrand[i]

fig = plt.figure(figsize=(6,3))
plt.plot(phi/np.pi, u0, 'r')
plt.plot(midpoint_phi/np.pi, np.max(u0), "*")
plt.ylabel("u0")

plt.twinx()
plt.plot(phi/np.pi, gh0)
plt.ylabel("gh")

fig = plt.figure(figsize=(6,3))
plt.plot(phi[25:32]/np.pi, gh0[25:32])


fig = plt.figure(figsize=(6,3))
plt.plot(phi/np.pi, integrand, 'g')
plt.ylabel("integrand")

plt.twinx()
plt.plot(phi/np.pi, gh0)
plt.ylabel("gh")

fig = plt.figure(figsize=(6,3))
plt.plot(phi/np.pi, integrand, 'g')
plt.ylabel("integrand")

plt.twinx()
plt.plot(phi/np.pi, silly_integration, 'y')
plt.ylabel("silly integration")

fig = plt.figure(figsize=(6,3))
plt.plot(phi/np.pi, integrand, 'g')
plt.ylabel("integrand")

plt.twinx()
plt.plot(phi/np.pi, u0, 'r')
plt.ylabel("u0")





In [ ]:
# The three different Coriolis models:

    

spheric = spheric_coriolis(phi)
const = const_coriolis(phi)
beta_model = beta_coriolis(phi)

fig = plt.figure(figsize=(6,3))
plt.plot(phi*180/np.pi, spheric, label="spheric")
plt.plot(phi*180/np.pi, const, label="const")
plt.plot(phi*180/np.pi, beta_model, label="beta")
plt.legend()
plt.title("Different Coriolis models")

print("max((spheric, const, beta_model)):", (spheric.max(), const.max(), beta_model.max()))
print("min((spheric, const, beta_model)):", (spheric.min(), const.min(), beta_model.min()))

print("midpoint_phi: ", midpoint_phi, spheric_coriolis(midpoint_phi))


#init_h_integral_beta  = lambda lat: RADIUS*init_u(lat)*(beta_coriolis(lat) + (np.tan(lat)/RADIUS)*init_u(lat))
#init_h_integral_const = lambda lat: RADIUS*init_u(lat)*(const_coriolis(lat) + (np.tan(lat)/RADIUS)*init_u(lat))
gh0_beta  = generate_gh0(phi, beta_coriolis, np.tan, phi_0, h_0_guestimate)
gh0_const = generate_gh0(phi, const_coriolis, np.tan, phi_0, h_0_guestimate)
gh0_beta_beta = generate_gh0(phi, beta_coriolis, beta_tan, phi_0, h_0_guestimate)

fig = plt.figure(figsize=(6,3))
plt.plot(phi*180/np.pi, gh0, label="spheric")
plt.plot(phi*180/np.pi, gh0_const, label="const")
plt.plot(phi*180/np.pi, gh0_beta, label="beta")
plt.plot(phi*180/np.pi, gh0_beta_beta, label="beta_beta")
plt.legend()
plt.title("h_init with different Coriolis models")

h_mean = np.mean(gh0_beta)
print("h_mean: ", h_mean)



## Finding the parameter $h_0$

It is defined such that the global mean of $h$ is 10000

In [ ]:
h_mean = 10000 # m

# integrand above phi_1:
double_int_above_phi_1 = scipy.integrate.quadrature(init_h_integrand, phi_0, phi_1, 
                                                    args=(beta_coriolis, beta_tan))[0]


def mean_integrand(lat, coriolis_function):
    return np.cos(lat)*scipy.integrate.quadrature(init_h_integrand, phi_0, lat, 
                                                  args=(coriolis_function, beta_tan))[0]


def mean_dbl_integrand(lat, y):
    return np.cos(y)*RADIUS*init_u(lat)*(beta_coriolis(lat) + (beta_tan(lat)/RADIUS)*init_u(lat))

def mean_dbl_integrand_above_phi_1(lat, y):
    return np.cos(y)*double_int_above_phi_1



print("double_int_above_phi_1: ", double_int_above_phi_1)
print("double_int_above_phi_1:  10658.96186949533 (with spherical)")


h_0_analytic = h_mean + (1.0/(2*GRAVITY))*double_int_above_phi_1*(1.0 - np.sin(phi_1) ) + \
               (1.0/(2*GRAVITY))*scipy.integrate.dblquad(mean_dbl_integrand, 
                                                         phi_0, phi_1, # np.pi/2.0,
                                                         lambda x: phi_0*np.ones_like(x), 
                                                         lambda x: x)[0]





print("h_0_guestimate: ", h_0_guestimate)
print("original h_0_analytic: 10152.969251376311")
print("h_0_analytic: ", h_0_analytic)

## Investigating initial conditions with Cartesian coordinates

In [ ]:
distance_between_latitudes = 111e3 # m
distance_equator_north_pole = 10e6

y_0   = distance_between_latitudes*phi_0*180/np.pi
y_1   = distance_between_latitudes*phi_1*180/np.pi
y_mid = distance_between_latitudes*midpoint_phi*180/np.pi


print ("phi_0, phi_1, phi_mid: ", (phi_0, phi_1, midpoint_phi))
print ("degrees_0, degrees_1, degrees_mid: ", (phi_0*180/np.pi, phi_1*180/np.pi, midpoint_phi*180/np.pi))
print ("y_0, y_1, y_mid: ", (y_0, y_1, y_mid))
print ("y_0_rel, y_1_rel, y_mid_rel: ", 
       (y_0/distance_equator_north_pole, y_1/distance_equator_north_pole, y_mid/distance_equator_north_pole))


In [ ]:
# We set the boundary in north and south to be at 10* and 80* north
y_south = 10*distance_between_latitudes
y_north = 80*distance_between_latitudes

ny = 200
dy = (y_north - y_south)/ny
dx = dy
print("(dx, dy): ", (dx, dy))


# Choose the domain to go around the earth (in x-direction)
distance_between_longitudes_45 = 78e3 # m
earth_circumsize_at_midpoint_phi = distance_between_longitudes_45*360
nx = np.floor(earth_circumsize_at_midpoint_phi/dx)
print("(nx, ny): ", (nx, ny))


In [ ]:
# Timestep
max_dt = 0.25*dx/(np.max(u0 + np.sqrt(gh0)))
dt = 0.8*max_dt
print("max_dt: ", max_dt)
print("dt: ", dt)

In [ ]:
# Our beta model compared to the beta model defined in phi

def our_beta_model(f, beta, cell_id, y_zero_reference_cell, dy):
    return f + beta * (cell_id - y_zero_reference_cell - 0.0)*dy


cells = np.linspace(0,ny-1, ny)
y_zero_reference_cell = (y_0 - y_south)/dy
f0 = spheric_coriolis(phi_0)
beta = (spheric_coriolis(phi_1)-spheric_coriolis(phi_0))/ (y_1 - y_0)  #df/dy
print("beta: ", beta)
print("y_zero_reference_cell: ", y_zero_reference_cell)

fig = plt.figure(figsize=(6,3))
plt.plot(distance_between_latitudes*phi*180/np.pi, beta_model, label="lat-based")
plt.plot(distance_between_latitudes*10 + cells*dy,
         our_beta_model(f0, beta, cells, y_zero_reference_cell, dy),
         label='y-based')
plt.legend()
plt.title(r'Different Coriolis $\beta$-models')



In [ ]:
# Define u and H with the above grid resulution 
# These are defined based on phi (with two ghost cells)
# and can be directly put into an array represented by y

phi_south = 10*np.pi/180 # 10 degrees
phi_north = 80*np.pi/180 # 80 degrees
dy_phi = (phi_north - phi_south)/ny
sim_phi = np.linspace(phi_south - 10*dy_phi, phi_north + 10*dy_phi, ny+20)
print("sim_phi.shape: ", sim_phi.shape)

sim_u_init = init_u(sim_phi)
print("sim_u_init.shape: ", sim_u_init.shape)

sim_gh_init = generate_gh0(sim_phi, beta_coriolis, beta_tan, phi_south, h_0_analytic)
sim_h_init = sim_gh_init/GRAVITY

sim_h_init_mean = np.mean(sim_h_init)
print("sim_h_init_mean: ", sim_h_init_mean)

sim_hu_init = sim_u_init*sim_h_init


# Calculate hu which is in geotrophic balance wrt sim_h_init (it's slope is equal to the slope of eta)
redef_hu_init = np.zeros_like(sim_h_init)
for j in range(1, len(redef_hu_init)-1):
    #y0_offset = 32.5
    y0_offset = 10.5 # Compensating for the open boundary, and location of cell centeres.
    redef_hu_init[j] = - (GRAVITY*sim_h_init_mean/our_beta_model(f0, beta, j, y_zero_reference_cell+y0_offset, dy))*\
                    (sim_h_init[j+1]-sim_h_init[j-1])/(2*dy)


fig = plt.figure(figsize=(6,3))
plt.plot(sim_phi/np.pi, sim_hu_init)
plt.ylabel("hu")

plt.twinx()
plt.plot(sim_phi/np.pi, sim_h_init, color='r')
plt.ylabel("h")


fig = plt.figure(figsize=(6,3))
plt.plot(sim_phi/np.pi, np.tan(sim_phi), label='tan')
plt.plot(sim_phi/np.pi, beta_tan(sim_phi), label='beta_tan')
beta_tan
plt.ylabel("tan")
plt.legend()


fig = plt.figure(figsize=(6,3))
plt.plot(sim_phi/np.pi, sim_hu_init, label='analytic')
plt.plot(sim_phi/np.pi, redef_hu_init, label='redefined')
plt.ylabel("hu")
plt.legend()




if False:
    h_syM_check = sim_h_init + sim_h_init[-1::-1]
    fig = plt.figure(figsize=(6,3))
    plt.plot(sim_phi/np.pi, h_syM_check, label='h sym check')
    plt.legend()

    u_syM_check = sim_hu_init - sim_hu_init[-1::-1]
    fig = plt.figure(figsize=(6,3))
    plt.plot(sim_phi/np.pi, u_syM_check, label='u_syM_check')
    plt.legend()

    redef_u_syM_check = redef_hu_init - redef_hu_init[-1::-1]
    fig = plt.figure(figsize=(6,3))
    plt.plot(sim_phi/np.pi, redef_u_syM_check, label='redef_u_syM_check')
    plt.legend()
    

# Perturbation function

In [ ]:
ghosts = np.array([10,2,10,2]) # north, east, south, west
dataShape = (int(ny)+ ghosts[0]+ghosts[2], int(nx)+ ghosts[1]+ghosts[3])
eta_pert = np.zeros(dataShape)

def perturb_eta(eta_pert, ghosts):
    nx = eta_pert.shape[1] - ghosts[1] - ghosts[3]
    ny = eta_pert.shape[0] - ghosts[0] - ghosts[2]
    
    radius_y_cells = distance_between_longitudes_45*180/dx
    mid_cell_y = int(eta_pert.shape[0]/2)
    mid_cell_x = int(nx/5) + ghosts[3]
    pert_alpha = 1/3
    pert_beta = 1/15
    h_hat = 120

    for j in range(ghosts[2], ny+ghosts[2]):
        for i in range(ghosts[3], nx+ghosts[3]):
            
            squared_dist_y = ((1/pert_beta)*(np.pi/180)*(j-mid_cell_y)*dy/distance_between_latitudes)**2
            squared_dist_x = ((1/pert_alpha)*(np.pi/180)*(i-mid_cell_x)*dx/(distance_between_longitudes_45))**2
            
            lat = np.cos(np.pi/4) # approximation into the beta-plane
            

            eta_pert[j,i] += h_hat*lat*np.exp(-squared_dist_y - squared_dist_x)
            
perturb_eta(eta_pert, ghosts)
fig = plt.figure(figsize=(18,6))
plt.imshow(eta_pert, origin='lower', interpolation='none')
plt.colorbar()

# Set up and animate simulations

In [ ]:

# Set initial conditions common to all simulators
sim_args = {
"gpu_ctx": gpu_ctx,
"nx": int(nx), "ny": int(ny),
"dx": dy, "dy": dy,
"dt": dt,
"g": GRAVITY,
"f": f0,
"coriolis_beta": beta,
"r": 0.0,
"y_zero_reference_cell": y_zero_reference_cell
}

eta_max = 1.5
hu_max = 1.5

In [ ]:

def sim_animation_old(simulator, T, crossection=False):
    eta1, u1, v1 = sim.download(interior_domain_only=True)
    
    #Create figure and plot initial conditions
    fig = plt.figure(figsize=(16, 16))
    domain_extent = [0, sim.nx*sim.dx, 0, sim.ny*sim.dy]
    
    ax_eta = plt.subplot(3,1,1)
    sp_eta = ax_eta.imshow(eta1, interpolation="none", origin='bottom', vmin=-eta_max, vmax=eta_max, extent=domain_extent)
    
    ax_u = plt.subplot(3,1,2)
    sp_u = ax_u.imshow(u1, interpolation="none", origin='bottom', vmin=-hu_max, vmax=hu_max, extent=domain_extent)
    
    ax_v = plt.subplot(3,1,3)
    sp_v = ax_v.imshow(v1, interpolation="none", origin='bottom', vmin=-hu_max, vmax=hu_max, extent=domain_extent)
    
    
    #Helper function which simulates and plots the solution
    def animate(i):
        if (i>0):
            t = sim.step(100*dt)
        else:
            t = 0.0
        eta1, u1, v1 = sim.download(interior_domain_only=True)
        
        #Update plots
        fig.sca(ax_eta)
        sp_eta.set_data(eta1)
        
        fig.sca(ax_u)
        sp_u.set_data(u1)
        
        fig.sca(ax_v)
        sp_v.set_data(v1)
        
        fig.suptitle("Time = {:04.0f} s ({:s})".format(t, sim.__class__.__name__), fontsize=18)
        print(".", end='')

    #Matplotlib for creating an animation
    anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
    plt.close(fig)
    return anim

In [ ]:
def sim_animation(simulator, T, plot_dt, crossection=False, spinUp=False):
    eta1, hu1, hv1 = sim.download(interior_domain_only=True)
    print("sim.download in sim_animator - eta1.shape: ", eta1.shape)
    
    #Create figure and plot initial conditions
    fig = plt.figure(figsize=(12, 12), constrained_layout=True)
    domain_extent = [0, eta1.shape[1]*sim.dx, 0, eta1.shape[0]*sim.dy]
    y_axis_eta = np.linspace(0.5*sim.dy, (sim.ny+0.5)*sim.dy, eta1.shape[0])
    y_axis_hu  = np.linspace(0.5*sim.dy, (sim.ny+0.5)*sim.dy, hu1.shape[0])
    y_axis_hv  = np.linspace(0.5*sim.dy, (sim.ny+0.5)*sim.dy, hv1.shape[0])
    mid_cell_x = int(eta1.shape[1]/2)
    pert_cell_x = int(eta1.shape[1]/5)
    
    gs = fig.add_gridspec(3,5)
    
    ax_eta = fig.add_subplot(gs[0,0:-2])
    sp_eta = ax_eta.imshow(eta1, interpolation="none", origin='bottom', vmin=-eta_max, vmax=eta_max, extent=domain_extent)
    
    ax_u = fig.add_subplot(gs[1,0:-2])
    sp_u = ax_u.imshow(hu1, interpolation="none", origin='bottom', vmin=-hu_max, vmax=hu_max, extent=domain_extent)
    
    ax_v = fig.add_subplot(gs[2,0:-2])
    sp_v = ax_v.imshow(hv1, interpolation="none", origin='bottom', vmin=-hu_max, vmax=hu_max, extent=domain_extent)
    
    if crossection:
        ax_eta_cs = fig.add_subplot(gs[0,-2:])
        cs_eta, =     ax_eta_cs.plot(y_axis_eta, eta1[:, mid_cell_x], '-' )
        cs_eta_init, = ax_eta_cs.plot(y_axis_eta, eta1[:, mid_cell_x], '--')
        cs_eta_pert, = ax_eta_cs.plot(y_axis_eta, eta1[:, pert_cell_x], '-.')
        ax_eta_cs.set_ylim(-eta_max*1.3, eta_max*1.3)
        ax_eta_cs.grid()
        
        ax_hu_cs = fig.add_subplot(gs[1,-2:])
        cs_hu, =     ax_hu_cs.plot(y_axis_hu, hu1[:, mid_cell_x], '-' )
        cs_hu_init, = ax_hu_cs.plot(y_axis_hu, hu1[:, mid_cell_x], '--')
        cs_hu_pert, = ax_hu_cs.plot(y_axis_hu, hu1[:, pert_cell_x], '-.')
        ax_hu_cs.set_ylim(-hu_max, hu_max)
        ax_hu_cs.grid()
        
        ax_hv_cs = fig.add_subplot(gs[2,-2:])
        cs_hv, =     ax_hv_cs.plot(y_axis_hv, hv1[:, mid_cell_x], '-' )
        cs_hv_init, = ax_hv_cs.plot(y_axis_hv, hv1[:, mid_cell_x], '--')
        cs_hv_pert, = ax_hv_cs.plot(y_axis_hv, hv1[:, pert_cell_x], '-.')
        ax_hv_cs.set_ylim(-hu_max, hu_max)
        ax_hv_cs.grid()
        
    #Helper function which simulates and plots the solution
    if spinUp:
        t = sim.step(20*plot_dt*dt)
    
    def animate(i):
        if (i>0):
            t = sim.step(plot_dt*dt)
        else:
            t = 0.0
        eta1, hu1, hv1 = sim.download(interior_domain_only=True)
        
        #Update plots
        fig.sca(ax_eta)
        sp_eta.set_data(eta1)
        
        fig.sca(ax_u)
        sp_u.set_data(hu1)
        
        fig.sca(ax_v)
        sp_v.set_data(hv1)
        
        if crossection:
            cs_eta.set_data(y_axis_eta, eta1[:, mid_cell_x])
            cs_hu.set_data(y_axis_hu,    hu1[:, mid_cell_x])
            cs_hv.set_data(y_axis_hv,    hv1[:, mid_cell_x])
            cs_eta_pert.set_data(y_axis_eta, eta1[:, pert_cell_x])
            cs_hu_pert.set_data(y_axis_hu,    hu1[:, pert_cell_x])
            cs_hv_pert.set_data(y_axis_hv,    hv1[:, pert_cell_x])
        
        fig.suptitle("Time = {:04.0f} s ({:s})".format(t, sim.__class__.__name__), fontsize=18)
        print("."+str(i)+".", end='')

    #Matplotlib for creating an animation
    anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
    plt.close(fig)
    return anim

In [ ]:
if 'sim' in globals():
    sim.cleanUp()
reload(CDKLM16)
    
ghosts = np.array([10,2,10,2]) # north, east, south, west
dataShape = (sim_args["ny"] + ghosts[0]+ghosts[2], 
             sim_args["nx"] + ghosts[1]+ghosts[3])

H = np.ones((dataShape[0]+1, dataShape[1]+1), dtype=np.float32) * sim_h_init_mean
eta0 = -np.ones(dataShape, dtype=np.float32) * sim_h_init_mean
u0 = np.zeros(dataShape, dtype=np.float32)
v0 = np.zeros(dataShape, dtype=np.float32)

print("sim_h_init.shape: ", sim_h_init.shape)
print("eta0.shape", eta0.shape)
print("(ny, nx): ", (sim_args["ny"], sim_args["nx"]))

for i in range(dataShape[1]):
    eta0[:,i] += sim_h_init
    u0[:,i] = redef_hu_init

eta_max = np.max(np.abs(eta0))
hu_max = np.max(np.abs(u0))*1.2
print("max eta and hu: ", (eta_max, hu_max))

#Create perturbation
perturb_eta(eta0, ghosts)

#Initialize simulator
cdklm_args = {"H": H, "eta0": eta0, "hu0": u0, "hv0": v0, "rk_order": 2,
              "boundary_conditions": Common.BoundaryConditions(3,2,3,2, spongeCells=[10,2,10,2])}
print("sim_args['y_zero_reference_cell']: ", sim_args['y_zero_reference_cell'])
sim = CDKLM16.CDKLM16(**cdklm_args, **sim_args)

#Run a simulation and plot it
#sim_animation(sim, T=20, plot_dt=100,  crossection=True)
sim_animation(sim, T=90, plot_dt=500, crossection=True, spinUp=True)

In [ ]:
if 'sim' in globals():
    sim.cleanUp()

ghosts = np.array([10,1,10,1]) # north, east, south, west
dataShape = (sim_args["ny"] + ghosts[0]+ghosts[2], 
             sim_args["nx"] + ghosts[1]+ghosts[3])

h0 = np.ones(dataShape, dtype=np.float32) * sim_h_init_mean
eta0 = -np.ones(dataShape, dtype=np.float32) * sim_h_init_mean;
u0 = np.zeros((dataShape[0], dataShape[1]+1), dtype=np.float32);
v0 = np.zeros((dataShape[0]+1, dataShape[1]), dtype=np.float32);  

print("sim_h_init.shape: ", sim_h_init.shape)
print("eta0.shape", eta0.shape)
print("(ny, nx): ", (sim_args["ny"], sim_args["nx"]))

for i in range(dataShape[1]):
    eta0[:,i] += sim_h_init
    u0[:,i] = redef_hu_init
    #u0[:,i] = sim_hu_init

eta_max = np.max(np.abs(eta0))
hu_max = np.max(np.abs(u0))*1.2
print("max eta and hu: ", (eta_max, hu_max))
    
#Create perturbation
perturb_eta(eta0, ghosts)
    
#Initialize simulator
ctcs_args = {"H": h0, "eta0": eta0, "hu0": u0, "hv0": v0, "A": 10000.0,
             "boundary_conditions": Common.BoundaryConditions(3,2,3,2, spongeCells=[10,1,10,1])}
sim = CTCS.CTCS(**ctcs_args, **sim_args)

#Run a simulation and plot it
#sim_animation(sim, T=20, plot_dt=100,  crossection=True)
sim_animation(sim, T=90, plot_dt=500, crossection=True, spinUp=True)

In [ ]:
crash

if 'sim' in globals():
    sim.cleanUp()

ghosts = [0,0,0,0] # north, east, south, west
dataShape = (sim_args["ny"] + ghosts[0]+ghosts[2], 
             sim_args["nx"] + ghosts[1]+ghosts[3])
    
h0 = np.ones(dataShape, dtype=np.float32) * 60
eta0 = np.zeros(dataShape, dtype=np.float32)
u0 = np.zeros((dataShape[0], dataShape[1]+1), dtype=np.float32)
v0 = np.zeros((dataShape[0]+1, dataShape[1]), dtype=np.float32)
    
#Create bump in to lower left of domain for testing
addCentralBump(eta0, sim_args["nx"], sim_args["ny"], sim_args["dx"], sim_args["dy"], ghosts)

#Initialize simulator
fbl_args = {"H": h0, "eta0": eta0, "hu0": u0, "hv0": v0}
sim = FBL.FBL(**fbl_args, **sim_args)

#Run a simulation and plot it
sim_animation(sim, T=50)

In [ ]:
if 'sim' in globals():
    sim.cleanUp()

ghosts = np.array([2,2,2,2]) # north, east, south, west
dataShape = (sim_args["ny"] + ghosts[0]+ghosts[2], 
             sim_args["nx"] + ghosts[1]+ghosts[3])

H = np.ones((dataShape[0]+1, dataShape[1]+1), dtype=np.float32) * 60.0
eta0 = np.zeros(dataShape, dtype=np.float32)
u0 = np.zeros(dataShape, dtype=np.float32)
v0 = np.zeros(dataShape, dtype=np.float32)

#Create bump in to lower left of domain for testing
addCentralBump(eta0, sim_args["nx"], sim_args["ny"], sim_args["dx"], sim_args["dy"], ghosts)

#Initialize simulator
kp_args = {"H": H, "eta0": eta0, "hu0": u0, "hv0": v0, "use_rk2": True}
sim = KP07.KP07(**kp_args, **sim_args)

#Run a simulation and plot it
sim_animation(sim, T=50)

In [ ]:
str(cuda.Context.get_cache_config())